# Exploring conformational space of selected macrocycles - "M1"

In this notebook we present and analyze selected structures, technical notes are [here](www.gitlab.com/user/gosia/icho).

In [47]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [48]:
import glob
import py3Dmol

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolAlign
from rdkit import rdBase
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())

2016.09.4
Wed Mar 29 16:08:16 2017


In [49]:
# Functions used in this notebook:
def grep_energies_from_sdf_outputs(files):
    energies = []
    for inp in files:
        with open(inp,'r') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                if "M  END" in line:
                    energies.append(float(lines[i+1]))
    return energies

### Crystal structure of "M1" macrocycle

In [50]:
cm1 = open('/home/gosia/work/work_on_gitlab/icho/calcs/m1/m1_crystal.xyz','r').read()
vcm1 = py3Dmol.view(width=400,height=400)
vcm1.removeAllModels()
vcm1.addModel(cm1,'xyz')
vcm1.setStyle({'stick':{'radius':0.15,'color':'spectrum'}})
vcm1.setBackgroundColor('0xeeeeee')
vcm1.zoomTo()
vcm1.show()

In [51]:
# decide what is the "core" - a part of molecule, which we wish to be most aligned (rmsd-wise) among all the structures
# here: pyridine ring
m1 = Chem.AddHs(Chem.MolFromSmiles('O=C1NCCNC(=O)c2nc(C(=O)NCCNC(=O)c3nc1ccc3)ccc2'))
core_m1 = m1.GetSubstructMatch(Chem.MolFromSmiles('n1ccccc1'))

In [52]:
templ_m1 = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_crystal.sdf')
m1_crystal = templ_m1[0]

### Conformers generated with the Balloon software:

Conformers were generated in two ways (genetic algorithm):

* Starting with the crystal geometry kept as a template, output: "m1_b_crystal" on the left fig. below

* Starting with the SMILES signature of M1 and allowing to "rebuild the geometry" (option --rebuildGeometry), output: "m1_b_smiles" on the right fig. below

In both cases the Balloon software was asked for 50 conformers (with other parameters set to default values) and it found 6 conformers in the former case and 7 in the latter.

In [53]:
inps_m1_b_sdf = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_crystalsdf/*.sdf')

In [54]:
inps_m1_b_smi = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/results_starting_from_crystalsmiles/*.sdf')

In [55]:
e_m1_b_sdf = grep_energies_from_sdf_outputs(inps_m1_b_sdf)
e_m1_b_smi = grep_energies_from_sdf_outputs(inps_m1_b_smi)

In [56]:
%%html
<table>
  <tr>
    <td id="m1_b_crystal" ></td>
    <td id="m1_b_smiles"  ></td>
  <tr>
    <td> m1_b_crystal </td>
    <td> m1_b_smiles  </td>  
  </tr>
</table>

,
m1_b_crystal,m1_b_smiles


In [57]:
# write conformers as a list
allmol_m1_b_sdf = []
allmol_m1_b_smi = []
suppl_m1_b_sdf  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_crystal_sdfout.sdf')
suppl_m1_b_smi  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_crystal_smilesout.sdf')

for mol in suppl_m1_b_sdf:
    allmol_m1_b_sdf.append(mol)
for mol in suppl_m1_b_smi:
    allmol_m1_b_smi.append(mol)    

In [58]:
# align:
for mol in allmol_m1_b_sdf:
    AllChem.AlignMolConformers(mol,atomIds=core_m1)
for mol in allmol_m1_b_smi:
    AllChem.AlignMolConformers(mol,atomIds=core_m1)    

In [59]:
# view:
p1_b_handles=[]

p1_b_sdf = py3Dmol.view(width=400,height=400)
for mol in allmol_m1_b_sdf:
    mb = Chem.MolToMolBlock(mol)
    p1_b_sdf.addModel(mb,'sdf')
p1_b_sdf.setStyle({'stick':{'radius':'0.15'}})
p1_b_sdf.setBackgroundColor('0xeeeeee')
p1_b_sdf.zoomTo()    
p1_b_handles.append(p1_b_sdf)

p1_b_smi = py3Dmol.view(width=400,height=400)
for mol in allmol_m1_b_smi:
    mb = Chem.MolToMolBlock(mol)
    p1_b_smi.addModel(mb,'sdf')
p1_b_smi.setStyle({'stick':{'radius':'0.15'}})
p1_b_smi.setBackgroundColor('0xeeeeee')
p1_b_smi.zoomTo()    
p1_b_handles.append(p1_b_smi)

In [60]:
p1_b_handles[0].insert('m1_b_crystal')

In [61]:
p1_b_handles[1].insert('m1_b_smiles')

Still, the conformers are very much alike. The energy (preoptimized with MM) and the RMSD calculated against the crystal structure is (for the "m1_b_crystal" and the "m1_b_smiles", respectively):

In [71]:
# calculate RMSD against the crystal structure /check that/:
for i, mol in enumerate(allmol_m1_b_sdf):
    name = "m1_b1_"+str(i)
    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(name, e_m1_b_sdf[i], AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(templ_m1[0]))))

name = m1_b1_0, E = 53.245621, RMS = 0.662122
name = m1_b1_1, E = 53.583766, RMS = 0.517415
name = m1_b1_2, E = 57.760058, RMS = 1.174442
name = m1_b1_3, E = 62.424468, RMS = 0.527526
name = m1_b1_4, E = 55.441273, RMS = 0.852633
name = m1_b1_5, E = 53.651385, RMS = 0.880096


In [73]:
# calculate RMSD against the crystal structure /check that/:
for i, mol in enumerate(allmol_m1_b_smi):
    name = "m1_b2_"+str(i)
    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(name, e_m1_b_smi[i], AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(templ_m1[0]))))

name = m1_b2_0, E = 53.280464, RMS = 0.756497
name = m1_b2_1, E = 53.729899, RMS = 1.205705
name = m1_b2_2, E = 54.259997, RMS = 0.894811
name = m1_b2_3, E = 57.332921, RMS = 0.408258
name = m1_b2_4, E = 63.187321, RMS = 0.382989
name = m1_b2_5, E = 54.028468, RMS = 0.770531
name = m1_b2_6, E = 53.919798, RMS = 1.239890


All conformers are very much alike (the program finds only *syn-syn* conformers), so we can choose for instance:

* m1_b1_0 (we reject: m1_b1_1, m1_b1_5, m1_b2_0, which have similar energy and RMS, hence probably will converge to the same minimum)
* m1_b1_2 
* m1_b1_3 (we reject m1_b2_4)
* m1_b2_2 (we reject m1_b1_4, m1_b2_4)
* m1_b2_3 (similar energy as m1_b1_2 but significantly different RMS)
* m1_b2_6 (similar energy as m1_b1_0 but significantly different RMS)

Below we will align the selected conformers:

In [80]:
selected_b = [allmol_m1_b_sdf[0], allmol_m1_b_sdf[2], allmol_m1_b_sdf[3], allmol_m1_b_smi[2], allmol_m1_b_smi[3], allmol_m1_b_smi[6]]

for mol in selected_b:
    #AllChem.AlignMolConformers(mol,atomIds=core_m1)
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles('n1ccccc1'))
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))
    
p_b = py3Dmol.view(width=400,height=400)
for mol in selected_b:
    mb = Chem.MolToMolBlock(mol)
    p_b.addModel(mb,'sdf')
p_b.setStyle({'stick':{'radius':'0.15'}})
p_b.setBackgroundColor('0xeeeeee')
p_b.zoomTo()
p_b.show()

### Conformers generated with the RDKit software:

RDKit found 9 conformers:

In [ ]:
inps_m1_rdkit = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/results_crystal_from_smiles_new/*.sdf')

In [ ]:
e_m1_rdkit = grep_energies_from_sdf_outputs(inps_m1_rdkit)

In [ ]:
# create a list of all structures to be aligned
allmol_m1_rdkit = []
suppl_m1_rdkit = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/rdkit/result_smiles_new.sdf')

for mol in suppl_m1_rdkit:
    allmol_m1_rdkit.append(mol)

This time, instead of visualizing all conformers, we will look directly at their alignment:

In [ ]:
# align:
for mol in allmol_m1_rdkit:
    AllChem.AlignMolConformers(mol,atomIds=core_m1)

In [ ]:
# view:
p = py3Dmol.view(width=400,height=400)
p.removeAllModels()
for mol in allmol_m1_rdkit:   
    mb = Chem.MolToMolBlock(mol)
    p.addModel(mb,'sdf')    
p.setStyle({'stick':{'radius':'0.15'}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()

In [ ]:
# calculate RMSD /check that/:
for mol in allmol_m1_rdkit:
    # note that the first structure on "allmol_m1_rdkit" list is the crystal structure, 
    # so the RMSD value calculated for the first structure (with respect to the crystal) will be 0 or almost 0
    print("Heavy Atom RMS:",AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(templ_m1[0])))

### Summary

* the Balloon software favors *syn-syn* conformers of amide groups (tested on various settings of the genetic algorithm implemented in that software)
* the RDKit software